<a href="https://colab.research.google.com/github/marcosfs2006/1oLabBoasPraticas/blob/master/Analise_Fluxo_Atuarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
library(readr)
library(ggplot2)
library(dplyr)
library(readxl)

# Análise do Fluxo Atuarial

O fluxo atuarial ser objeto de análise está contido no arquivo `2019_FLX_CIVIL_PREV_GA_XXXXXXXXXXXXXX.csv`. 

Para que seja possível realizar a análise, serão necessários dois outros arquivos:  

* `2019_FLX_CIVIL_PREV_GA_cnpj - Com cálculo Duração.xlsx` 
* `Comparativo de Receita Despesa.csv`   

Estes dois arquivos estão disponíveis no site da SPrev.

# Importação dos dados

Para realizar a análise dos dados é necessário realizar a importação dos mesmos, o que faremos a seguir. 



In [0]:
# Gerar um data frame com os nomes e os códigos das varíaveis

dicionario <- read_excel("/content/2019_FLX_CIVIL_PREV_GA_cnpj - Com cálculo Duração.xlsx", sheet="Flx_CIVIL_PREV_GA", range="A7:BQ8")
dicionario <- t(dicionario)
dicionario <- data.frame(cod = dimnames(dicionario)[[1]], 
                       descr = unname(dicionario[, 1]))
dicionario$cod <- paste("V", dicionario$cod, sep = "")

head(dicionario)

cod,descr
<chr>,<fct>
V100101,Instante
V100201,Ano
V100301,Taxa de Juros (%)
V100401,Fator de Desconto
V109001,Base de Cálculo da Contribuição Normal
V111000,Benefícios Concedidos - Contribuições dos Aposentados


Vamos definir uma pequena função que calculará a o **Fator de Desconto** (`V100401`) a partir das **Taxas de Juros** (`V100301`) informadas no fluxo atuarial.

In [0]:
# Define uma função auxiliar para calcular as taxas de desconto
#--------------------------------------------------------------
calcula_tx_desconto <- function(i, postecipado = TRUE){
  #--------------------
  # i: Variável V100301
  #--------------------
  if(postecipado){
    desconto <- 1 / (1 + i)
  } else {
    desconto <- c(1, 1 / (1 + i[2:length(i)])) 
  }
  round(cumprod(desconto), 5)
}

In [0]:
# Importação dos dados
flx_orig <- read_csv2("/content/2019_FLX_CIVIL_PREV_GA_XXXXXXXXXXXXXX.csv", skip = 5, col_names = FALSE)

# Coloca nomes nas colunas
colfluxo <- c(100101, 100201, 100301, 100401, 109001,	111000,	111101,	111201, 111301,
              111401,	112000,	119900,	121000,	121100,	121200,	121300,	121400,	121500,
              121600,	121700,	122000, 122100,	122200,	122300,	122400,	122500,	122600,
              122700,	123000,	123100,	123200,	123300,	123400, 124000,	124100,	124200,
              124300,	124400,	124500,	129000,	130101,	130201,	139901,	190000,	210000,
              211001,	212001,	213001,	214001,	215001,	219901,	220000,	221000,	222000,
              223000,	224000,	225000, 226000,	227000,	229000,	239901,	240000,	250001,
              260001,	270001,	280001,	290001)

if(ncol(flx_orig) == 67){
  names(flx_orig) <- paste("V", colfluxo, sep="")
} else { 
  names(flx_orig) <- paste("V", c(colfluxo, 310000,	320000), sep="")
}

